In [1]:
from langchain_anthropic import ChatAnthropic
claude_api_key=""
llm = ChatAnthropic(model="claude-3-5-sonnet-20240620",api_key=claude_api_key)

In [4]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["AI", "Law", "Social", "Other"] = Field(
        ...,
        description="Given a user question choose to route it to AI or Law or Social or other.",
    )

def query_router(query):
    structured_llm_router = llm.with_structured_output(RouteQuery)
    
    system = """You are an expert at routing a user question to a vectorstore or web search or casual conversation.
    The vectorstore contains documents related to language processing, and artificial intelligence.
    Use the vectorstore for questions on these topics. For all else, use web-search.
    If the question is a casual conversation use original knowledge"""
    route_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system),
            ("human", "{question}"),
        ]
    )
    
    question_router = route_prompt | structured_llm_router
    result=question_router.invoke({"question": query})

    return result

query="2022년부터 2023년 사이에 가장 주목 받은 LLM 모델들에 대해 설명해주세요. "
query_router(query)

RouteQuery(datasource='AI')